<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import env as E
from buffers import ReplayBuffer, nStepBuffer
from agent import D4PG_Agent
import models

# from agent import Agent
#from get_args import get_args

import os.path
import time
import re
import sys

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T

In [2]:
import multiprocessing as multi
multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [3]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        
args = Args()

<b>Confirm that the args are all set the way we want them.

In [4]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000


## Load the environment
<i> & print a bit of information contained in the wrapper class

In [5]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [6]:
env.train = False
env.reset()

In [11]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(20):
    actions = np.random.randn(env.agent_count, env.action_size)
    actions = np.clip(actions, -1, 1)
    rewards, dones = env.step(actions)
    scores += rewards
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.0


### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [ ]:
# agent = agent.D4PG_Agent(env.state_size, env.action_size, env.agent_count, args.rollout)
# print(agent.__class__.__name__)
# print(agent.memory)
# agent.initialize_memory(10, env)

In [194]:
env.reset()
states = env.states

### Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [190]:
actor = models.ActorNet(env.state_size, env.action_size)
env.reset()

In [33]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    actions = actor(torch.from_numpy(states).float()).detach().numpy()
    print("ACTIONS:", actions)
    rewards, dones = env.step(actions)
    scores += rewards
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

ACTIONS: [[ 0.11488349  0.32546467 -0.02945378 -0.11915834]
 [ 0.03125562  0.3307379   0.01640427 -0.1419087 ]
 [ 0.10339975  0.2186998   0.15593217  0.03585169]
 [ 0.19228022  0.33946118  0.01146774 -0.12320453]
 [ 0.09295374  0.22028081  0.15397821  0.02675368]
 [ 0.09891359  0.31160975  0.04120457 -0.08115304]
 [ 0.04515991  0.28131548  0.088663   -0.05282905]
 [ 0.03108213  0.32326168  0.02803557 -0.1641748 ]
 [ 0.04564152  0.31911698  0.02541842 -0.11746107]
 [-0.00055805  0.2995411  -0.01914931 -0.1661469 ]
 [ 0.01922878  0.3302283   0.01729521 -0.16410424]
 [ 0.17923522  0.33214998  0.00606754 -0.12154298]
 [ 0.05545576  0.29136026  0.06001689 -0.0505356 ]
 [ 0.07760954  0.29918164 -0.02387793 -0.1362928 ]
 [ 0.21038446  0.35189664  0.01533572 -0.12330478]
 [ 0.03533383  0.3254104   0.02108988 -0.14290836]
 [ 0.07664435  0.22639188  0.16475186  0.03012107]
 [ 0.06970263  0.26032576  0.14959191  0.01880245]
 [ 0.03638733  0.32437164  0.01899731 -0.13360079]
 [ 0.0877688   0.22810

 [ 0.03727947  0.19927481  0.20755489  0.1241722 ]]
ACTIONS: [[ 0.19283874  0.28907236  0.1925274  -0.15055218]
 [-0.00088407  0.2978985   0.08880787 -0.10616143]
 [ 0.05117374  0.20536397  0.20346282  0.13551398]
 [ 0.16495435  0.25431785  0.2013706  -0.15600473]
 [ 0.04680557  0.20354164  0.21339881  0.12723184]
 [ 0.07949049  0.23885593  0.13057572  0.02157764]
 [ 0.02082977  0.1929256   0.15665898  0.09431793]
 [ 0.02719114  0.27430758  0.06849325  0.01385995]
 [ 0.03115785  0.289407    0.05864344  0.006067  ]
 [ 0.08164813  0.24790254  0.1453167  -0.09420349]
 [ 0.02406199  0.30168676  0.05924575 -0.09037574]
 [ 0.17471386  0.24068317  0.21221606 -0.13377327]
 [ 0.03701174  0.24360555  0.08495409  0.06809826]
 [ 0.11033973  0.26156422  0.18252774 -0.1140562 ]
 [ 0.17062119  0.26746055  0.17206357 -0.14434245]
 [ 0.00872176  0.31348845  0.06620266 -0.0625499 ]
 [ 0.04064302  0.26785836  0.1751923   0.08149415]
 [ 0.05256815  0.24496357  0.20776033  0.10132312]
 [ 0.01677289  0.3012

### Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [189]:
import importlib
import agent
import buffers
importlib.reload(agent)
importlib.reload(buffers)
importlib.reload(models)

<module 'models' from 'C:\\Users\\mattdoll\\Development\\deep-reinforcement-learning\\p2_continuous-control\\models.py'>

In [191]:
critic = models.CriticNet(env.state_size, env.action_size)
critic.forward

<bound method CriticNet.forward of CriticNet(
  (fc1): Linear(in_features=33, out_features=400, bias=True)
  (fc2): Linear(in_features=404, out_features=300, bias=True)
  (logits): Linear(in_features=300, out_features=128, bias=True)
)>

In [192]:
print(critic.fc1.weight)
print(critic.fc2.weight)
print(critic.logits.weight)
print(critic.fc1.bias)

Parameter containing:
tensor([[-3.2218e-04,  1.6935e-03, -2.6910e-03,  ..., -7.5556e-06,
         -1.8317e-03,  2.2504e-03],
        [ 1.9928e-03, -6.3402e-04, -3.1496e-04,  ..., -2.6411e-04,
          1.4683e-03,  1.9111e-03],
        [ 1.1278e-03,  1.5864e-03, -2.8928e-04,  ..., -1.8489e-04,
          2.6820e-03,  2.6257e-03],
        ...,
        [ 4.2443e-04, -1.0562e-03,  1.6833e-03,  ..., -2.7642e-03,
          2.7230e-03,  1.8398e-03],
        [ 2.4794e-03,  2.2959e-03,  1.5051e-03,  ..., -2.5648e-04,
         -1.4497e-04, -1.3992e-03],
        [ 6.7582e-04, -2.1176e-03,  5.4849e-04,  ..., -7.9557e-04,
          6.4881e-04, -4.6626e-05]], requires_grad=True)
Parameter containing:
tensor([[-7.4623e-04,  2.0983e-03, -2.3317e-03,  ...,  2.1025e-03,
         -2.8000e-03, -6.7842e-04],
        [ 1.5082e-03, -2.7357e-03,  2.5440e-03,  ..., -2.2641e-03,
         -9.4961e-04, -1.1625e-03],
        [ 2.1419e-03, -1.5581e-03,  2.5888e-03,  ..., -2.5120e-03,
          1.4424e-03,  1.2140e-

In [193]:
actor.fc1.weight

Parameter containing:
tensor([[ 1.9650e-03,  2.3277e-03, -2.2439e-03,  ..., -8.6732e-04,
         -2.3603e-03, -1.5881e-03],
        [ 2.0762e-03, -1.3740e-03, -1.3019e-03,  ...,  4.7480e-05,
         -2.7765e-03,  1.9084e-03],
        [-1.7923e-03,  8.1489e-04, -2.3946e-04,  ...,  1.9119e-03,
          2.7611e-03,  1.7337e-03],
        ...,
        [ 2.0558e-04, -2.9391e-03, -1.6046e-03,  ...,  1.4433e-04,
         -8.1171e-04, -3.6432e-04],
        [ 2.9259e-03,  2.2537e-03, -4.8642e-04,  ..., -8.8945e-04,
         -2.9430e-03, -2.4668e-03],
        [-2.5740e-03,  1.3579e-03, -4.0553e-05,  ..., -2.6403e-03,
         -2.9557e-03,  2.8189e-03]], requires_grad=True)

In [196]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    states = torch.from_numpy(states).float()
    actions = actor(states).detach().numpy()
    rewards, dones = env.step(actions)
    scores += rewards
    values = critic(states, torch.from_numpy(actions))
    #print(critic.Q_val)
    #print(values.sample())
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.0


In [197]:
critic.y

tensor([[0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        ...,
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078]],
       grad_fn=<SoftmaxBackward>)

In [198]:
critic.z

tensor([[0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        ...,
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
        [0.0078, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078]],
       grad_fn=<SoftmaxBackward>)

In [ ]:
env.close()